In [8]:
import os
import re

In [5]:
# Get the list of all files and directories
path = r"C:\Users\gxb18167\OneDrive - University of Strathclyde\Desktop\PhD\2nd year\Results\best"
dir_list = os.listdir(path)
print(dir_list)

['Augment_v2_1_ablation_noise_random_task1_task2_taskNRv2_finetune_DCGAN_v1_Text_skipstep1_b32_20_30_5e-05_5e-07_unique_sent+Sentence_Level+.txt', 'Augment_v2_1_ablation_noise_TF-IDF-High_task1_task2_taskNRv2_finetune_DCGAN_v1_Text_skipstep1_b32_20_30_5e-05_5e-07_unique_sent+Sentence_Level+.txt', 'Augment_v2_1_ablation_noise_TF-IDF-Low_task1_task2_taskNRv2_finetune_DCGAN_v1_Text_skipstep1_b32_20_30_5e-05_5e-07_unique_sent+Sentence_Level+.txt', 'Augment_v2_1_ablation_noise_TF-IDF-Medium_task1_task2_taskNRv2_finetune_DCGAN_v1_Text_skipstep1_b32_20_30_5e-05_5e-07_unique_sent+Sentence_Level+.txt', 'Augment_v2_1_random_task1_task2_taskNRv2_finetune_DCGAN_v1_Text_skipstep1_b32_20_30_5e-05_5e-07_unique_sent+Contextual+.txt', 'Augment_v2_1_random_task1_task2_taskNRv2_finetune_DCGAN_v1_Text_skipstep1_b32_20_30_5e-05_5e-07_unique_sent+Sentence_Level+.txt', 'Augment_v2_1_random_task1_task2_taskNRv2_finetune_DCGAN_v1_Text_skipstep1_b32_20_30_5e-05_5e-07_unique_sent+Word_Level+.txt', 'Augment_v2_1_

In [7]:
f = open(rf"{path}\{dir_list[0]}", "r")
for x in f:
  print(x)

corpus BLEU-1 score: 0.13306426602893723

corpus BLEU-2 score: 0.039765216532574976

corpus BLEU-3 score: 0.01704272210598092

corpus BLEU-4 score: 0.008155510666275852

ROUGE score: {'rouge-1': {'r': 0.1186501222788642, 'p': 0.1702998372028562, 'f': 0.13456138605893508}, 'rouge-2': {'r': 0.02232341933760975, 'p': 0.026131443944172567, 'f': 0.022119511324491605}, 'rouge-l': {'r': 0.11434612580538803, 'p': 0.16211379667771927, 'f': 0.12902622747464865}}



In [17]:
import ast

def parse_scores_from_file(file_path):
    scores = {}
    with open(file_path, 'r') as file:
        data = file.read()

        # Corpus BLEU scores
        bleu_scores = re.findall(r'corpus BLEU-(\d) score: (\d+\.\d+)', data)
        for n, score in bleu_scores:
            scores[f'BLEU-{n}'] = float(score)

        # ROUGE scores
        rouge_scores_match = re.search(r'ROUGE score: ({.+})', data)
        if rouge_scores_match:
            rouge_scores_str = rouge_scores_match.group(1)
            scores['ROUGE'] = ast.literal_eval(rouge_scores_str)

    return scores

In [18]:
scores = parse_scores_from_file(rf"{path}\{dir_list[0]}")

In [19]:
scores

{'BLEU-1': 0.13306426602893723,
 'BLEU-2': 0.039765216532574976,
 'BLEU-3': 0.01704272210598092,
 'BLEU-4': 0.008155510666275852,
 'ROUGE': {'rouge-1': {'r': 0.1186501222788642,
   'p': 0.1702998372028562,
   'f': 0.13456138605893508},
  'rouge-2': {'r': 0.02232341933760975,
   'p': 0.026131443944172567,
   'f': 0.022119511324491605},
  'rouge-l': {'r': 0.11434612580538803,
   'p': 0.16211379667771927,
   'f': 0.12902622747464865}}}

In [20]:
import pandas as pd

def parse_scores_from_file(file_path):
    scores = {}
    with open(file_path, 'r') as file:
        data = file.read()

        # Corpus BLEU scores
        bleu_scores = re.findall(r'corpus BLEU-(\d) score: (\d+\.\d+)', data)
        for n, score in bleu_scores:
            scores[f'BLEU-{n}'] = float(score)

        # ROUGE scores
        rouge_scores_match = re.search(r'ROUGE score: ({.+})', data)
        if rouge_scores_match:
            rouge_scores_str = rouge_scores_match.group(1)
            rouge_scores = ast.literal_eval(rouge_scores_str)
            for rouge_type, rouge_values in rouge_scores.items():
                for metric, value in rouge_values.items():
                    scores[f'{rouge_type}-{metric}'] = value

    return scores

In [21]:
scores = parse_scores_from_file(rf"{path}\{dir_list[0]}")

In [22]:
df = pd.DataFrame([scores])

In [23]:
df

,BLEU-1,BLEU-2,BLEU-3,BLEU-4,rouge-1-r,rouge-1-p,rouge-1-f,rouge-2-r,rouge-2-p,rouge-2-f,rouge-l-r,rouge-l-p,rouge-l-f
0,0.133064,0.039765,0.017043,0.008156,0.11865,0.1703,0.134561,0.022323,0.026131,0.02212,0.114346,0.162114,0.129026
